In [101]:
import statistics
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate , train_test_split , StratifiedShuffleSplit, cross_val_score
from sklearn import   linear_model, metrics
from sklearn.linear_model import SGDClassifier as SGD

## Кодируем категориальный столбец по среднему значению real_feature (счетчик)

In [14]:
def code_mean(data, category_feature, real_feature):
    return (data[category_feature].map(data.groupby(category_feature)[real_feature].mean()))

In [84]:
train_directory = "data/train.csv"

train = pd.read_csv(train_directory, header=0 )

train['y'] = [1 if y == 'yes' else 0 for y in train['y']]

In [85]:
train.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
count,3222.000000,3222.000000,3222.000000,3222.000000,3222.000000,3222.00000,3222.000000,3222.000000,3222.000000,3222.000000,3222.000000
mean,39.949100,254.391682,2.525140,961.075729,0.191186,0.09162,93.587576,-40.580261,3.618885,5166.177654,0.112353
std,10.292639,253.617387,2.546888,190.390725,0.552781,1.55203,0.578841,4.575879,1.734232,74.028498,0.315849
min,18.000000,0.000000,1.000000,0.000000,0.000000,-3.40000,92.201000,-50.800000,0.635000,4963.600000,0.000000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.80000,93.075000,-42.700000,1.334000,5099.100000,0.000000
50%,38.000000,181.000000,2.000000,999.000000,0.000000,1.10000,93.876000,-41.800000,4.857000,5191.000000,0.000000
75%,47.000000,314.000000,3.000000,999.000000,0.000000,1.40000,93.994000,-36.400000,4.961000,5228.100000,0.000000
max,88.000000,3643.000000,35.000000,999.000000,6.000000,1.40000,94.767000,-26.900000,5.045000,5228.100000,1.000000


### Кодирование занятости (Job)

In [86]:
train['job'].value_counts()

admin.           796
blue-collar      707
technician       525
services         309
management       251
retired          127
self-employed    123
entrepreneur     117
unemployed        92
housemaid         78
student           64
unknown           33
Name: job, dtype: int64

In [87]:
train['job'] = code_mean(train, 'job', 'y')

### Попытка закодировать marital -> результат оказался хуже

In [88]:
train['marital'].value_counts()

married     1966
single       904
divorced     341
unknown       11
Name: marital, dtype: int64

In [107]:
married_dict = {
    'married' : 1,
    'single' : 0,
    'divorced' : 0,
    'unknown' : 0
}
train['marital'] = train['marital'].replace(married_dict)

In [109]:
train['marital']

0       1
1       0
2       1
3       1
4       1
       ..
3217    1
3218    1
3219    1
3220    0
3221    0
Name: marital, Length: 3222, dtype: int64

In [90]:
"""train['marital'] = code_mean(train, 'marital', 'y')"""

"train['marital'] = code_mean(train, 'marital', 'y')"

In [120]:
train['education'].value_counts()

0.133130    984
0.104972    724
0.077951    449
0.124390    410
0.095385    325
0.080214    187
0.160839    143
Name: education, dtype: int64

In [115]:
train['education'] = code_mean(train, 'education', 'y')

In [123]:
train['housing'].value_counts()

yes        1703
no         1434
unknown      85
Name: housing, dtype: int64

In [132]:
yesNoUnknown_dict = {
    'yes' : 1,
    'no' : 0,
    'unknown' : 0.5
}

In [124]:
train['housing'] = train['housing'].replace(yesNoUnknown_dict)

In [130]:
train['loan'].value_counts()

no         2619
yes         518
unknown      85
Name: loan, dtype: int64

In [133]:
train['loan'] = train['loan'].replace(yesNoUnknown_dict)

In [139]:
train['contact'].value_counts()

cellular     2068
telephone    1154
Name: contact, dtype: int64

In [141]:
contact_dict = {
    'cellular' : 1,
    'telephone' : 0
}
train['contact'] = train['contact'].replace(contact_dict)

In [177]:
train['poutcome'].value_counts()

nonexistent    2762
failure         348
success         112
Name: poutcome, dtype: int64

In [178]:
poutcome_dict = {
    'success' : 1,
    'failure' : 0,
    'nonexistent' : 0.5
}
train['poutcome'] = train['poutcome'].replace(poutcome_dict)

In [183]:
train['month'] = code_mean(train, 'month', 'y')
train['day_of_week'] = code_mean(train, 'day_of_week', 'y')
train['default'] = code_mean(train, 'default', 'y')

In [184]:
train.corr().style.background_gradient(cmap='coolwarm')

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
age,1.000000,0.114818,0.272152,-0.012404,-0.166931,-0.013016,-0.003459,-0.024893,0.079665,0.002644,0.039014,-0.006501,-0.040607,0.044000,0.023187,-0.011676,0.008798,0.086147,-0.009824,-0.036183,0.053425
job,0.114818,1.000000,-0.139783,0.213015,0.089736,0.026758,-0.018023,0.080394,0.133314,0.008102,0.046961,0.001973,-0.114460,0.073341,0.030362,-0.108708,-0.057062,0.106243,-0.111394,-0.148203,0.132422
marital,0.272152,-0.139783,1.000000,-0.108133,-0.124140,-0.018213,0.010302,-0.086077,-0.053270,-0.020822,-0.018403,-0.002855,0.030034,-0.049362,0.022322,0.075652,0.044833,0.047485,0.083334,0.077083,-0.048132
education,-0.012404,0.213015,-0.108133,1.000000,0.153753,0.019541,-0.006666,0.104655,0.084061,0.007730,0.013619,0.019422,-0.069735,0.054596,0.036519,-0.058161,-0.075213,0.144568,-0.051299,-0.076447,0.072262
default,-0.166931,0.089736,-0.124140,0.153753,1.000000,0.013851,-0.018879,0.157966,0.103763,-0.011027,0.017884,-0.017215,-0.091164,0.095020,0.000103,-0.187312,-0.164665,-0.024002,-0.180920,-0.172365,0.068977
housing,-0.013016,0.026758,-0.018213,0.019541,0.013851,1.000000,0.074346,0.072651,0.028530,-0.004576,-0.027865,-0.014165,-0.029167,0.031169,0.011670,-0.047265,-0.087947,-0.020665,-0.044255,-0.021994,-0.009219
loan,-0.003459,-0.018023,0.010302,-0.006666,-0.018879,0.074346,1.000000,-0.007430,0.010276,-0.006014,-0.012153,-0.011468,0.003842,0.002791,0.002385,-0.004323,-0.016446,0.005521,0.003590,0.009879,-0.011874
contact,-0.024893,0.080394,-0.086077,0.104655,0.157966,0.072651,-0.007430,1.000000,0.114543,0.038983,0.029472,-0.094052,-0.121818,0.217410,-0.128400,-0.385557,-0.566559,-0.264540,-0.393114,-0.265316,0.132529
month,0.079665,0.133314,-0.053270,0.084061,0.103763,0.028530,0.010276,0.114543,1.000000,0.042655,0.020216,-0.056582,-0.229516,0.242125,0.000522,-0.312333,-0.096542,0.085249,-0.320178,-0.397902,0.242612
day_of_week,0.002644,0.008102,-0.020822,0.007730,-0.011027,-0.004576,-0.006014,0.038983,0.042655,1.000000,-0.004550,0.010307,-0.000392,0.005599,0.004193,-0.024159,-0.003513,-0.046153,-0.031299,-0.029671,0.027150


### По полученной таблице корреляции можно выделить следующие столбцы, имеющие коэффициент корреляции с Y больше 0.2 по модулю:
#### month
#### duration
#### pdays
#### previous
#### emp.var.rate
#### nr.employed (брать euribor3m может не иметь смысла из-за крайне высокой корреляции с nr.employed)

### Также были взяты столбцы с корреляцией больше 0,12, которые при ручном добавлении увеличивали качество модели:
#### contact
#### age
#### job
#### poutcome

In [228]:
# Определяем X и y
X = train[['previous', 'duration' , 'pdays', 'emp.var.rate', 'nr.employed', 'month', 'contact','age', 'job', 'poutcome']][:]
y = train['y'][:]

In [230]:
X.corr().style.background_gradient(cmap='coolwarm')

,previous,duration,pdays,emp.var.rate,nr.employed,month,contact,age,job,poutcome
previous,1.000000,0.025395,-0.590461,-0.409944,-0.520039,0.242125,0.217410,0.044000,0.073341,-0.274019
duration,0.025395,1.000000,-0.045726,-0.028857,-0.046760,0.020216,0.029472,0.039014,0.046961,0.026701
pdays,-0.590461,-0.045726,1.000000,0.271116,0.390379,-0.229516,-0.121818,-0.040607,-0.114460,-0.481153
emp.var.rate,-0.409944,-0.028857,0.271116,1.000000,0.892328,-0.312333,-0.385557,-0.011676,-0.108708,0.197385
nr.employed,-0.520039,-0.046760,0.390379,0.892328,1.000000,-0.397902,-0.265316,-0.036183,-0.148203,0.124426
month,0.242125,0.020216,-0.229516,-0.312333,-0.397902,1.000000,0.114543,0.079665,0.133314,0.000522
contact,0.217410,0.029472,-0.121818,-0.385557,-0.265316,0.114543,1.000000,-0.024893,0.080394,-0.128400
age,0.044000,0.039014,-0.040607,-0.011676,-0.036183,0.079665,-0.024893,1.000000,0.114818,0.023187
job,0.073341,0.046961,-0.114460,-0.108708,-0.148203,0.133314,0.080394,0.114818,1.000000,0.030362
poutcome,-0.274019,0.026701,-0.481153,0.197385,0.124426,0.000522,-0.128400,0.023187,0.030362,1.000000


In [144]:
classifier = RandomForestClassifier()

In [229]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
statistics.mean(cross_validate(classifier,  X, y, cv=sss, scoring='roc_auc')['test_score'])

0.9203606748109366

In [237]:
from sklearn.model_selection import RandomizedSearchCV
# count of trees in random forest
n_estimators = [100, 150, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200]#, 1000, 1200, 1400, 1600, 1800, 2000
# count of features for splitting
max_features = ['auto', 'sqrt', 'log2']
# max number of levels in tree
max_depth = [2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100, None]#, 40, 50, 60, 70, 80, 90, 100
# min of samples for splitting
min_samples_split = [2, 3, 5, 8, 10, 12, 15, 20, 25, 40, 60, 80, 150, 200, 250, 300, 350]
# min of samples for a leaf
min_samples_leaf = [1, 2, 3, 5, 7, 10, 15, 20, 25, 30, 40, 50, 60, 100]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Loss criteria
#criterion = ['mse', 'mae']

parameters = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}#, 'criterion': criterion, 'max_features': max_features

rf_random = RandomizedSearchCV(estimator = classifier, param_distributions = parameters, 
                               n_iter = 200, cv = sss, verbose=2, scoring = 'roc_auc', random_state=30)
rf_random.fit(X, y)

print(rf_random.best_params_)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
[CV] n_estimators=1800, min_samples_split=15, min_samples_leaf=20, max_features=auto, max_depth=7, bootstrap=True 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=1800, min_samples_split=15, min_samples_leaf=20, max_features=auto, max_depth=7, bootstrap=True, total=   9.0s
[CV] n_estimators=1800, min_samples_split=15, min_samples_leaf=20, max_features=auto, max_depth=7, bootstrap=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.9s remaining:    0.0s


[CV]  n_estimators=1800, min_samples_split=15, min_samples_leaf=20, max_features=auto, max_depth=7, bootstrap=True, total=   9.2s
[CV] n_estimators=1800, min_samples_split=15, min_samples_leaf=20, max_features=auto, max_depth=7, bootstrap=True 
[CV]  n_estimators=1800, min_samples_split=15, min_samples_leaf=20, max_features=auto, max_depth=7, bootstrap=True, total=   9.2s
[CV] n_estimators=1800, min_samples_split=15, min_samples_leaf=20, max_features=auto, max_depth=7, bootstrap=True 
[CV]  n_estimators=1800, min_samples_split=15, min_samples_leaf=20, max_features=auto, max_depth=7, bootstrap=True, total=   9.2s
[CV] n_estimators=1800, min_samples_split=15, min_samples_leaf=20, max_features=auto, max_depth=7, bootstrap=True 
[CV]  n_estimators=1800, min_samples_split=15, min_samples_leaf=20, max_features=auto, max_depth=7, bootstrap=True, total=   9.5s
[CV] n_estimators=400, min_samples_split=10, min_samples_leaf=30, max_features=log2, max_depth=100, bootstrap=False 
[CV]  n_estimators

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed: 70.6min finished


{'n_estimators': 150, 'min_samples_split': 12, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 7, 'bootstrap': True}


In [234]:
classifierNew = RandomForestClassifier(n_estimators=800, min_samples_split=10, min_samples_leaf=10, max_features='log2', max_depth=40, bootstrap=True)
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=30)
statistics.mean(cross_validate(classifierNew,  X, y, cv=sss, scoring='roc_auc')['test_score'])

0.931064572425829

In [236]:
classifierNew = RandomForestClassifier(n_estimators=1200, min_samples_split=10, min_samples_leaf=2, max_features='auto', max_depth=8, bootstrap=True)
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=30)
statistics.mean(cross_validate(classifierNew,  X, y, cv=sss, scoring='roc_auc')['test_score'])

0.931151832460733

In [239]:
classifierNew = RandomForestClassifier(n_estimators=150, min_samples_split=12, min_samples_leaf=2, max_features='sqrt', max_depth=7, bootstrap=True)
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=30)
statistics.mean(cross_validate(classifierNew,  X, y, cv=sss, scoring='roc_auc')['test_score'])

0.9300659298041497